In [2]:
from sklearn import svm
import numpy as np
#data = [open, high, low, close, volume USD, EMA difference], most recent entries first
data_newest = np.loadtxt(open("data/Bitstamp_Minutely.csv", "rb"), delimiter=",", skiprows=2, usecols=(3,4,5,6,8))

#oldest entries first data
X = data_newest[::-1]

def get_EMA(x, short=9, long=26, column=0):
    EMA = np.zeros((len(x),3))     #Returns EMA matrix with entires: [short period EMA, long period EMA, difference]
    
    #** REFER TO ABOVE MARKDOWN CELL
    EMA[long-1][0] = np.sum(x[long-short:long])/short   #These are SMAs used for the intial EMA calculation
    EMA[long-1][1] = np.sum(x[0:long])/long             #These are SMAs used for the intial EMA calculation

    #calculate EMA:
    w0 = 2/(1+short)   #short period weights
    w1 = 2/(1+long)    #long period weights
    for i in range(long, len(x)):
        EMA[i][0] = x[i]*w0 + EMA[i-1][0]*(1-w0)    #short term EMA 
        EMA[i][1] = x[i]*w1 + EMA[i-1][1]*(1-w1)    #long term EMA 
        EMA[i][2] = EMA[i][0]-EMA[i][1]
    return EMA

def attach_column(M, c):
    if M.shape[0] != len(c):
        print("Error: Cannot attach column to matrix with different sizes")
        return
    M = np.insert(M, M.shape[1], c, axis=1)
    return M

EMA = get_EMA(X[:,0])

X = attach_column(X, EMA[:,2])
X = X[27:]
y = np.zeros(X.shape[0])

for i in range(len(X)-15):
    y[i] = np.sign(np.average(X[i:i+5,0])-X[i][0])

In [8]:
C_vals = [.01, .1, 1, 10, 100]

accuracies = []
for c in C_vals:
    model = svm.SVC(C=c, kernel='rbf')
    model.fit(X[:50000], y[:50000])
    accuracies += [model.score(X[50000:51000],y[50000:51000])]

In [9]:
accuracies

[0.476, 0.477, 0.477, 0.535, 0.586]

In [ ]:
[0.562, 0.562, 0.557, 0.558, 0.555]